## Twitter Data Research

---

### Question: 
Each job_id in the twitter data has a 'description' tied to it.  Can we predict what the 'description' of the tweet is based on the text?

##### First, I load some python libraries, and choose which job_ids I want to analyze.  I then create a connection to the database.*

***note** I ended up connecting to the subset database because the main database kept timing out

In [1]:
import numpy as np
import pandas as pd
import pymysql as mdb

allTweets = {}

topics = {
    1160: "#tornado",
    2577: "#worldseries",
    2616: "autism",
    4244: "alabama",
    4245: "#istandwithloftin",
    4246: "@istandwithloftin",
    4259: "PorteOuverte", 
    4253: "mizzouunited", 
    4255: "concernedstudent1950", 
    4258: "prayformizzou",
    4257: "#MIZZOU"
}

allDescriptions = {
    1160: "weather",
    2577: "baseball",
    2616: "autism",
    4244: "Florida Hurricane",
    4245: "Loftin",
    4246: "Loftin",
    4259: "Paris 2015", 
    4253: "Mizzou", 
    4255: "Mizzou", 
    4258: "Mizzou",
    4257: "Mizzou"
}

connection = mdb.connect('128.206.116.195', 'tg4_ro', '?3stEt7!3hUbRa-R', 'tw4_db')
subsetConnection = mdb.connect('opendata.missouri.edu', 'datascience', 'datascience', 'datascience')

##### Next, I went through and collected the job_id and tweet text from all of the tweets in each job.

In [2]:
for jobId, topic in topics.items():
    sql = 'SELECT job_id, text FROM tweet WHERE job_id = ' + str(jobId)
    allTweets[jobId] = pd.read_sql(sql, subsetConnection)
    print(topic)

#worldseries
prayformizzou
PorteOuverte
alabama
#istandwithloftin
@istandwithloftin
autism
#tornado
#MIZZOU
mizzouunited
concernedstudent1950


In [8]:
for key in allTweets:
    print(str(key) + " " + str(len(allTweets[key])) + " " + allDescriptions[key])

2577 2527317 baseball
4258 201833 Mizzou
4259 354430 Paris 2015
4244 7273542 Florida Hurricane
4245 160 Loftin
4246 0 Loftin
4257 755779 Mizzou
2616 7495015 autism
1160 168882 weather
4253 2269 Mizzou
4255 248646 Mizzou


##### Now, I need to create my training array and my test array.  The training array will hold the tweets and descriptions that I am going to use to create my model.  This should allow me to run the test array with my model and produce a high accuracy of predictions.

In [9]:
training = []
test = []

for key in allTweets:
    total = len(allTweets[key])
    count = 0
    
    description = allDescriptions[key]
    
    breakingPoint = total*(3/4)
    
    if (total != 0):
        print(total)
        for key2, value2 in allTweets[key].iterrows():
            temp = {
                    'job_id' : allTweets[key].loc[key2, 'job_id'],
                    'text' : allTweets[key].loc[key2, 'text'],
                    'description' : description
            }
            
            if (count < breakingPoint):
                training.append(temp)
            else:
                test.append(temp)
            count = count + 1
            if (count % 100000 == 0):
                print(count)
        print("Done with: " + str(total))

2527317
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
Done with: 2527317
201833
100000
200000
Done with: 201833
354430
100000
200000
300000
Done with: 354430
7273542
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
Done with: 7273542
160
Done with: 160
755779
100000
200000
300000
400000
500000
600000
700000
Done with: 755779
7495015
10

##### Documentation for the python Naive Bayes Library can be found [here](https://github.com/muatik/naive-bayes-classifier).

In [17]:
from naiveBayesClassifier import tokenizer
from naiveBayesClassifier.trainer import Trainer
from naiveBayesClassifier.classifier import Classifier

trainer = Trainer(tokenizer)

##### With the training and test sets complete, I can now import the Python naive Bayes library and train my model.

In [18]:
for item in training:
    trainer.train(item['text'], item['description'])

In [19]:
classifier = Classifier(trainer.data, tokenizer)

##### With my model trained, I can now use my test set to see how accurate my model is with predicting the description based on the tweet text.

In [22]:
correctPredicted = 0

for item in test:
    res = classifier.classify(item['text'])
    if res[0][0] == item['description']:
        correctPredicted += 1

In [24]:
print(correctPredicted)
print(len(test))
print('\n' + str(round(correctPredicted/len(test)*100,2)) + "% of all of the test tweets were predicted correctly")

4504352
4756964

94.69% of all of the test tweets were predicted correctly


###### The data set ended up being 94.69% correct!